# Vectrix Graphs

### Packages needed to run this notebook
- libmagic: ```brew install libmagic```

### ChromaDB
Don't forget to start the ChromaDB server: ```docker run -p 7777:8000 chromadb/chroma```

### Run the API
https://docs.astral.sh/uv/guides/integration/fastapi/

For development purposes, you can run the API with: ```uv run fastapi dev```

In [1]:
# Load packages from the src directory
import sys
import json
sys.path.append('../src')
from vectrix_graphs import ExtractDocuments, setup_logger, ExtractMetaData


from dotenv import load_dotenv
load_dotenv()

2024-11-06 16:00:10,487 - VectorDB - WARNING - Connecting using localhost
2024-11-06 16:00:10,544 - VectorDB - WARNING - Demo collection already exists


True

### Extracting chunks of data from a document

In [2]:
# Create chunks of data from a document
extract = ExtractDocuments(
    logger=setup_logger(name="Files", level="INFO"),
    )

result = extract.extract(file_path="./files/attention_is_all_you_need.pdf")


2024-11-06 15:50:18,458 - Files - INFO - Extracting documents from ./files/attention_is_all_you_need.pdf


In [4]:
print('Metadata:')
print(json.dumps(result[1].metadata, indent=4))

print('Content:')
print(result[1].page_content)

Metadata:
{
    "file_directory": "./files",
    "filename": "attention_is_all_you_need.pdf",
    "languages": [
        "eng"
    ],
    "last_modified": "2024-10-22T21:44:24",
    "page_number": 1,
    "orig_elements": "eJx1VE2P2zYQ/SsDnS11rdV+eI8Btrn0I0Ad5LBYGLQ0kohSpEJSttUg/71vKK3bpsnJ5pAz8+a9p3n5krHhgW086CZ7ouyG7+qWlcqrcrvLq4fdTa4q5nzXbOumrur7Bz5mG8oGjqpRUSHnS1Y75xttVeSQzkbNboqHnnXXR0QediVS1uhZN7FH8H4rwdFpGyXr5WVb3Rb3G6q2t0X5uqG3893DXfEo5+p+Vzz+/7y8RyALc4g8yBQf9IXNH6OqOfuKi1YbPjTacx2dn+VB8ZPEQrZeWjWwhFWMoEI7e9DhoIw5APHBMjfF2LRZmsF2k+rSnC8Z2y57TdEQD4NrdKs5sVjelFW+vcnLcl9un6rqqawke0TmwU7DkT1ebQVa5IswlO17psYNINFGCvx5YlszRa9saKZaIBEasAmkPNNRBW4IsdoNo+ELYbTJe2An5xG0J2cmSVKGLE8+/cSz838Gir2KpG1tpoZJWUIjFPb425CihtOpIAF05BBpZN86D2TdFYEJTppY8Ily/J8SawHEvZu6XjpcWaWB615ZHYaCPjGN3o0uAATAnSloGeUNJ8asex3RYfK8SV32QoZAYb9ZGQjOsJmFiO/0CBtqdBjZBsF+1rG/0lRzwvovogJJuke1gp4vmFnLRxGkNPDQoIDGroIYlRpFFUBn6N1Z4GGOlZ7oQByFSYpADW3p86SMjjOde3gNlwuX0HFUkAYj6L/U0SyYPLTXXl4E3Vk4qoYjMCPcitJAJfUBQ9uCfp/80pQEHp84UP

In [5]:
# Add additional metadata using a NER-pipeline
ner = ExtractMetaData(
    logger=setup_logger(name="ExtractMetaData", level="INFO",),
    model="gpt-4o-mini" # Options are gpt-4o-mini, llama3.1-8B , llama3.1-70B
    )
result_with_metadata = ner.extract(result, source="uploaded_file")

2024-11-06 15:50:32,617 - ExtractMetaData - INFO - Extracting metadata from 52 documents, using gpt-4o-mini


In [6]:
result_with_metadata[0].metadata

{'filename': 'attention_is_all_you_need.pdf',
 'filetype': 'application/pdf',
 'author': 'Ashish Vaswani',
 'source': 'uploaded_file',
 'word_count': 107,
 'language': 'EN',
 'content_type': 'other',
 'tags': "['AI', 'Research', 'Machine Learning']",
 'summary': "The document discusses the paper titled 'Attention Is All You Need', which is associated with Google Brain and Google Research, and includes multiple authors from these institutions and the University of Toronto.",
 'read_time': 0.535,
 'last_modified': '2024-10-22T21:44:24'}

### Adding the documents to a vector database (Chroma)

For this demo, the vector database will be saved locally on disk, restarting the container will delete the database.
I prefer using the cosine distance instead of the default squared L2 distance, we pass this using the `hnsw:space` metadata.

$$
d = 1.0 - \frac{\sum(A_i \times B_i)}{\sqrt{\sum(A_i^2) \cdot \sum(B_i^2)}}
$$

We use Ollama to calculate the embeddings locally with BGE-M3, since over a 100 langues are supported this is ideal for embedding Arabic documents.

BGE-M3 is based on the XLM-RoBERTa architecture and is distinguished for its versatility in Multi-Functionality, Multi-Linguality, and Multi-Granularity:

- Multi-Functionality: It can simultaneously perform the three common retrieval functionalities of embedding model: dense retrieval, multi-vector retrieval, and sparse retrieval.
- Multi-Linguality: It can support more than 100 working languages.
- Multi-Granularity: It is able to process inputs of different granularities, spanning from short sentences to long documents of up to 8192 tokens.

> ℹ️ So all embeddings will be calculated locally ℹ️


In [3]:
from vectrix_graphs import vectordb

vectordb.remove_collection("demo")
vectordb.create_collection("demo")

In [7]:
vectordb.add_documents(result_with_metadata)

In [2]:
from vectrix_graphs import vectordb
# Now let's query the vector database
vectordb.similarity_search(
    query="What is the attention mechanism?",
    k=3
    )

[[0.0209198, -0.015304565, 0.022125244, 0.00919342, -0.02003479, -0.029891968, 0.014839172, -0.0068359375, 0.04006958, 0.0440979, -0.011634827, -0.03845215, 0.023834229, -0.017471313, -0.04937744, -0.018188477, 0.008811951, -0.0005168915, 0.027786255, 0.010368347, 0.010620117, 0.023620605, 0.03527832, -0.024276733, 0.03111267, -0.012779236, 0.0030727386, -0.0035476685, -0.009437561, -0.022521973, 0.015281677, 0.006542206, 0.042663574, -0.019012451, -0.0022125244, 0.008239746, 0.014572144, -0.02772522, -0.020065308, 0.020874023, -0.01159668, 0.028671265, 0.031921387, -0.025115967, -0.0055770874, 0.014953613, -0.04449463, -0.052978516, 0.04776001, 0.010391235, 0.008453369, 0.018447876, 0.012023926, -0.004611969, -0.01965332, -0.007820129, -0.064941406, 0.026641846, 0.016921997, 0.05291748, 0.023071289, 0.02709961, -0.012512207, -0.018096924, 0.0047721863, 0.004951477, 0.05026245, 0.007949829, -0.017211914, -0.024551392, 0.00091028214, 0.009010315, 0.036132812, -0.030899048, -0.03289795, 

Exception: {"error":"InvalidArgumentError","message":"Expected where document to have exactly one operator, got {}"} (trace ID: 0)

In [9]:
import chromadb

embeddings = [[0.0209198, -0.015304565, 0.022125244, 0.00919342, -0.02003479, -0.029891968, 0.014839172, -0.0068359375, 0.04006958, 0.0440979, -0.011634827, -0.03845215, 0.023834229, -0.017471313, -0.04937744, -0.018188477, 0.008811951, -0.0005168915, 0.027786255, 0.010368347, 0.010620117, 0.023620605, 0.03527832, -0.024276733, 0.03111267, -0.012779236, 0.0030727386, -0.0035476685, -0.009437561, -0.022521973, 0.015281677, 0.006542206, 0.042663574, -0.019012451, -0.0022125244, 0.008239746, 0.014572144, -0.02772522, -0.020065308, 0.020874023, -0.01159668, 0.028671265, 0.031921387, -0.025115967, -0.0055770874, 0.014953613, -0.04449463, -0.052978516, 0.04776001, 0.010391235, 0.008453369, 0.018447876, 0.012023926, -0.004611969, -0.01965332, -0.007820129, -0.064941406, 0.026641846, 0.016921997, 0.05291748, 0.023071289, 0.02709961, -0.012512207, -0.018096924, 0.0047721863, 0.004951477, 0.05026245, 0.007949829, -0.017211914, -0.024551392, 0.00091028214, 0.009010315, 0.036132812, -0.030899048, -0.03289795, -0.003364563, 0.026947021, 0.0028514862, 0.02848816, -0.028717041, 0.008895874, -0.0025043488, 0.004737854, 0.020584106, -0.014190674, 0.03869629, -0.007820129, -0.019882202, 0.026382446, -0.057800293, -0.012832642, 0.025375366, -0.0017967224, -0.05215454, -0.07287598, -0.05291748, -0.02394104, -0.04788208, -0.0005660057, 0.025512695, -0.025772095, 0.0010690689, 0.025299072, 0.00091934204, -0.017456055, -0.022994995, -0.007663727, -0.022720337, -0.01209259, 0.04800415, -0.035339355, -0.052978516, 0.0075187683, 0.023132324, -0.006515503, -0.023223877, -0.052886963, 0.06768799, -0.01576233, 0.044647217, 0.00036215782, 0.014419556, -0.046325684, -0.012306213, 0.007598877, -0.054534912, -0.041992188, 0.052947998, 0.06561279, -0.012664795, -0.012069702, -0.00283432, -0.019332886, 0.010307312, -0.006164551, -0.019714355, 0.048980713, -0.028320312, -0.013046265, -0.015357971, 0.057434082, 0.012641907, 0.013374329, 0.0030727386, -0.023040771, 0.029846191, 0.0335083, -0.007637024, -0.050201416, -0.05038452, -0.037384033, 0.040374756, 0.033599854, -0.008491516, -0.04901123, -0.02897644, -0.07922363, -0.0007452965, 0.023391724, -0.03466797, 0.081970215, -0.020217896, -0.025985718, 0.043029785, -0.028335571, 0.035308838, -0.05429077, -0.0061149597, 0.0010824203, 0.028137207, -0.013114929, 0.040985107, -0.056610107, -0.008552551, 0.0087509155, -0.0061035156, -0.029174805, 0.03765869, 0.010017395, -0.005554199, 0.010803223, -0.057159424, -0.005126953, 0.031097412, 0.0016794205, 0.038513184, 0.0018568039, 0.07092285, 0.00041747093, 0.030471802, -0.032073975, 0.010116577, -0.01689148, -0.03074646, -0.044158936, -0.0021877289, 0.003030777, 0.03149414, -0.013534546, 0.028305054, 0.04425049, -0.005428314, 0.013900757, -0.071899414, -0.03253174, -0.027923584, -0.007873535, 0.028518677, 0.010063171, 0.020507812, 0.0005440712, -0.0012817383, -0.0039138794, -0.048675537, -0.05105591, -0.019729614, -0.013153076, 0.02798462, -0.018722534, 0.0181427, -0.057800293, 0.001250267, -0.05142212, -0.018035889, 0.05899048, -0.0078125, 9.864569e-05, -0.002708435, 0.03933716, 0.028747559, -0.044525146, -0.00869751, -0.07531738, -0.018615723, -0.025650024, -0.013267517, -0.0035991669, -0.015617371, 0.035858154, 0.02633667, -0.0066223145, 0.05026245, -0.021697998, 0.061462402, 0.0927124, 0.03488159, 0.009284973, 0.035858154, -0.01979065, 0.011474609, -0.028930664, 0.00724411, 0.012054443, -0.06359863, 0.011413574, 0.029800415, 0.0010671616, 0.017608643, -0.020309448, -0.010101318, 0.021621704, -0.0011720657, -0.020141602, 0.005241394, 0.00034213066, -0.06088257, -0.01890564, -0.03643799, 0.013641357, -0.044311523, 0.03137207, -0.06982422, -0.011886597, -0.019165039, 0.021209717, -0.043762207, -0.01600647, 0.014602661, -0.010505676, -0.03552246, 0.0057907104, 0.042938232, -0.0026626587, -0.025939941, 0.030349731, 0.034362793, -0.034606934, -0.024368286, -0.020462036, 0.0289917, 0.057159424, 0.032684326, 0.024795532, -0.026412964, -0.006549835, 0.007545471, 0.021621704, -0.012084961, 0.032196045, -0.039001465, -0.00504303, 0.048614502, -0.024414062, 0.026931763, 0.008552551, -0.008934021, -0.0032463074, 0.04840088, 0.008956909, -0.0020370483, -0.035614014, 0.020751953, -0.007785797, -0.025054932, -0.05682373, -0.013557434, -0.075805664, -0.041259766, -0.089416504, 0.03527832, 0.040100098, -0.005771637, 0.0256958, -0.01159668, -0.0060768127, -0.027145386, 0.017654419, -0.0018568039, -0.04168701, 0.02848816, 0.009483337, 0.0021476746, -0.028869629, 0.023132324, -0.015060425, 0.031951904, 0.023269653, -0.010246277, 0.0069732666, 0.0035171509, 0.06384277, -0.040161133, 0.0022354126, 0.0051460266, -0.016586304, 0.006324768, 0.013183594, 0.01802063, -0.022369385, 0.020462036, -0.024215698, -0.04055786, -0.045440674, 0.009391785, 0.015350342, -0.011428833, 0.03475952, 0.006134033, 0.0029449463, -0.0491333, 0.022323608, 0.04940796, -0.02520752, -0.08935547, 0.027053833, -0.06896973, -0.024124146, -0.048553467, 0.05444336, 0.03552246, -0.00081825256, 0.03503418, 0.0027713776, 0.016937256, 0.02848816, -0.048919678, 0.0054626465, -0.0284729, 0.013267517, -0.008407593, -0.010734558, 0.021743774, 0.00548172, 0.025253296, 0.08428955, -0.006046295, 0.004043579, 0.016189575, 0.045196533, 0.03111267, -0.02458191, -0.05117798, -0.06011963, -0.005290985, -0.021469116, -0.009422302, 0.019500732, -0.027923584, -0.040130615, -0.024337769, -0.00894928, 0.035461426, -0.006706238, 0.035125732, -0.0748291, 0.10424805, -0.00969696, 0.0065727234, 0.040740967, -0.0049362183, 0.022521973, -0.042663574, 0.015411377, -0.05529785, -0.016036987, -0.016052246, 0.036987305, -0.014289856, 0.031234741, -0.049926758, 0.008590698, -0.02519226, 0.051818848, 0.028671265, 0.04321289, 0.07055664, -0.051086426, -0.07531738, 0.03338623, 0.03338623, 0.007549286, 0.028259277, -0.029037476, -0.003791809, -0.032287598, 0.033691406, 0.008239746, 0.007827759, 0.040130615, -0.018371582, -0.042541504, -0.014694214, -0.02960205, 0.02458191, -0.028961182, 0.0038700104, -0.042175293, -0.03665161, -0.050231934, -0.037872314, -0.023620605, 0.012825012, 0.019042969, 0.04248047, 0.019256592, 0.089904785, -0.041778564, 0.00088500977, 0.0703125, 0.024291992, 0.006587982, 0.009132385, 0.00920105, -0.00091552734, 0.0088272095, -0.03817749, 0.010261536, 0.010231018, -0.016357422, 0.016159058, 0.010322571, 0.029891968, -0.015655518, -0.028030396, -0.020492554, -0.028686523, 0.039886475, -0.016433716, 0.05807495, -0.024276733, 0.02458191, 0.006462097, 0.0178833, 0.0104522705, 0.021057129, 0.00058078766, -0.025665283, 0.013214111, -0.009979248, -0.019241333, 0.039245605, -0.0231781, 0.01890564, 0.013404846, -0.02319336, -0.05090332, 0.0061035156, -0.009132385, 0.026443481, -0.023590088, 0.008010864, 0.021026611, -0.000623703, -0.029541016, -0.00504303, 0.025558472, 0.01777649, -0.02078247, -0.0067596436, -0.026870728, 0.018508911, -0.048187256, 0.074157715, -0.0003402233, -0.00077676773, 0.015571594, -0.0027694702, 0.014785767, -0.0036640167, -0.005168915, -0.0007867813, 0.0006532669, 0.015960693, 0.020721436, -0.009208679, -0.012542725, 0.0038452148, 0.016937256, -0.022262573, -0.033721924, 0.038208008, 0.020584106, 0.012924194, 0.02532959, -0.018127441, 0.036315918, 0.002735138, 0.020568848, 0.07122803, 0.009155273, 0.007408142, -0.01133728, -0.022232056, 0.011299133, -0.028244019, 0.055480957, 0.007724762, 0.028305054, 0.027709961, -0.07122803, 0.026626587, 0.052520752, -0.006965637, -0.0072517395, 0.021499634, -0.11340332, -0.05834961, 0.0023002625, 0.01637268, 0.05230713, 0.054473877, 0.010528564, 0.01007843, -0.03186035, 0.013793945, 0.009124756, -0.02406311, -0.094055176, -0.05267334, 0.0034103394, -0.05126953, -0.009567261, -0.09869385, -0.06121826, -0.034210205, 0.037719727, -0.029922485, 0.005393982, -0.012649536, 0.034729004, 0.038635254, 0.056732178, -0.04837036, 0.04547119, -0.018356323, 0.012008667, 0.012550354, -0.00730896, -0.025283813, -0.004558563, 0.029846191, -0.0074539185, -0.013000488, -0.058654785, -0.009155273, 0.022918701, -0.027374268, -0.01071167, 0.025665283, -0.013893127, 0.0036754608, 0.011253357, 0.01309967, -0.0017023087, -0.0071868896, 0.004047394, -0.026641846, 0.012306213, -0.02822876, 0.022949219, 0.030807495, 0.015411377, 0.026748657, -0.036834717, 0.039855957, 0.009735107, -0.023773193, 0.04019165, 0.029556274, -0.022918701, 0.022262573, 0.0056381226, 0.0019683838, 0.02079773, 0.025283813, 0.0028133392, -0.015899658, 0.03186035, 0.012954712, 0.0011911392, 0.023208618, -0.013465881, -0.059906006, -0.08093262, -0.04949951, 0.06365967, -0.025253296, -0.031829834, -0.01763916, -0.050354004, 0.015029907, -0.0006136894, -0.008811951, -0.0007843971, -0.016677856, -0.016677856, -0.008361816, 0.0018320084, 0.0033874512, -0.00046038628, 0.0012817383, 0.00046277046, -0.007282257, -0.025680542, 0.0025749207, -0.004501343, -0.052947998, 0.001121521, -0.011619568, -0.032165527, -0.035461426, 0.027740479, 0.0129852295, 0.046691895, 0.042022705, 0.06097412, 0.012374878, 0.048675537, -0.0048103333, 0.027572632, -0.003446579, -0.005947113, 0.00617218, -0.035614014, 0.027404785, 0.036224365, -0.007835388, 0.013877869, 0.04171753, -0.047821045, 0.0010757446, -0.027511597, 0.018188477, -0.027008057, -0.015167236, 0.022094727, 0.022277832, 0.010932922, 0.029525757, 0.0058174133, -0.012863159, 0.005420685, -0.0022506714, -0.00076818466, 0.044555664, 0.036010742, -0.04449463, 0.0029010773, -0.0027999878, 0.04031372, 0.030044556, -0.019638062, -0.010498047, -0.04336548, 0.0055503845, 0.0056648254, 0.0043296814, -0.038726807, 0.037750244, 0.02670288, -0.023239136, -0.036712646, 0.013427734, -0.0075302124, 0.010765076, -0.0134887695, 0.022583008, 0.0038852692, -0.017028809, 0.032287598, 0.006752014, -0.009567261, 0.041625977, 0.01637268, 0.030578613, 0.013877869, -0.005870819, 0.0317688, -0.0049705505, -0.022506714, 0.026412964, 0.02935791, -0.011741638, 0.015464783, 0.008049011, 0.016448975, -0.012107849, 0.02748108, 0.026107788, -0.024795532, -0.038726807, -0.03237915, 0.030441284, 0.020950317, 0.00919342, 0.051971436, 0.035888672, -0.015472412, -0.021011353, 0.0020923615, -0.021240234, -0.029769897, -0.08282471, -0.01675415, 0.016235352, -0.058807373, -0.07446289, 0.008766174, -0.045959473, 0.033325195, -0.037017822, 0.014862061, 0.008956909, -0.00022923946, -0.030395508, -0.0065460205, -0.026306152, 0.005836487, -0.04067993, -0.0046691895, -0.0026569366, -0.010818481, -0.020904541, -0.014671326, 0.036590576, 0.027770996, -0.0008816719, 0.057739258, 0.024108887, -0.047302246, -0.05316162, 0.04043579, 0.058288574, 0.019638062, -0.0011959076, 0.019714355, 0.005290985, 0.015731812, -0.017822266, -0.013549805, 0.018218994, 0.036499023, 0.058288574, 0.01651001, 0.035858154, 0.010444641, -0.014411926, 0.007972717, -0.01826477, -0.009544373, 0.059020996, -0.015220642, 0.00434494, -0.022583008, -0.032287598, -0.047058105, 0.014923096, 0.018356323, -0.013572693, 0.0075569153, -0.009361267, -0.013153076, 0.006919861, -0.038909912, -0.01222229, -0.023101807, -0.0052604675, -0.00944519, -0.023147583, -0.04348755, 0.04537964, 0.059173584, -0.008934021, 0.004207611, -0.031280518, 0.012084961, -0.014099121, -0.016784668, 0.019470215, 0.04333496, 0.0071525574, 0.015525818, 0.09906006, 0.06921387, 0.07647705, -0.016479492, -0.0140686035, 0.02357483, -0.028671265, -0.030899048, 0.035491943, -0.002292633, 0.03488159, 0.027954102, 0.018493652, 0.048065186, 0.02268982, -0.056274414, -0.014518738, 0.09063721, -0.017364502, 0.015213013, 0.0814209, -0.0082473755, -0.00041270256, -0.009895325, 0.0021839142, -0.023391724, 0.01576233, 0.020065308, -0.040100098, 0.038482666, 0.034973145, 0.060150146, -0.015548706, -0.03274536, -0.018539429, -0.035339355, 0.024734497, -0.08276367, 0.0023841858, 0.031799316, 0.06402588, 0.038024902, -0.0071258545, 0.06112671, 0.02166748, -0.018066406, 0.062561035, 0.0046653748, 0.01713562, -0.0034179688, 0.0050315857, -0.052246094, -0.027023315, 0.012374878, -0.009048462, 0.013038635, 0.006793976, 0.0035324097, 0.0013933182, -0.015151978, 0.011665344, 0.015487671, 0.03869629, 0.03656006, 0.0066375732, -0.0039749146, -0.0009698868, -0.033599854, 0.031143188, -0.00082302094, 0.031143188, -0.022888184, -0.0014314651, 0.017211914, 0.046966553, -0.0007648468, -0.0044555664, -0.030166626, -0.025512695, -0.0055236816, 0.0031661987, 0.058380127, -0.022644043, -0.059906006, -0.031311035, 0.008049011, 0.0021915436, -0.037078857, -0.06036377, -0.01499176, 0.031280518, 0.08093262, 0.035461426, -0.07092285, 0.1005249, 0.03741455, -0.038513184, -0.011230469, -0.013870239, -0.019561768, 0.01914978, -0.0026130676, 0.013458252, -0.0149383545, -0.030456543, -0.0022201538, -0.0050468445, -0.017837524, -0.018997192, -0.0259552, 0.014091492, 0.032196045, -0.008613586, 0.028869629, 0.0446167, -0.04977417, -0.029449463, -0.02748108, -0.008094788, 0.00166893, -0.012458801, -0.008529663, 0.010070801, 0.038848877, -0.030670166, -0.050323486, -0.017593384, -0.012702942, 0.0066604614, -0.007385254, 0.021026611, 0.052703857, 0.026931763, 0.0090789795, -0.004840851, -0.054504395, 0.0012369156, -0.0035114288, -0.023788452, -0.019424438, 0.0013074875, -0.03314209, -0.0008659363, -0.008422852, 0.0076789856, -0.023147583, 0.012466431, -0.020584106, -0.017196655, -0.0065345764, -0.051635742, 0.007843018, -0.026168823, 0.014526367, -0.039794922, -0.06695557, 0.033843994, 0.016616821, -0.05557251, 0.029373169, -0.023239136, 0.022277832, -0.0008378029, 0.0107803345, 0.013137817, -0.004009247, -0.008338928, 0.022506714, 0.02407837, 0.015945435, -0.0006327629, 0.0055274963, -0.0044555664, 0.02128601, -0.02897644, 0.06695557, -0.0028572083, 0.021774292, -0.011787415, 0.043548584, -0.044921875, -0.016036987, -0.03439331, -0.031097412, 0.0036067963, 0.030197144, 0.0446167, -0.03515625, 0.023834229, -0.05496216, 0.032318115, 0.010124207]]


client = chromadb.HttpClient(host='localhost', port=7777)

collection = client.get_collection("demo")
collection.query(
    query_embeddings=embeddings
    where=None
)

Exception: {"error":"InvalidArgumentError","message":"Expected where to have exactly one operator, got {}"} (trace ID: 0)

## Asking questions to the Graph
 Let's now ask questions using the LangGraph workflow

### Example 1: Using closed source LLMs


### Example 2: Using open-source LLMs that can be self-hosted

In [ ]:
# Load packages from the src directory
import sys
from IPython.display import Markdown, display, Image
sys.path.append('../src')

from langchain_core.messages import HumanMessage
from vectrix_graphs import local_slm_demo

# Display the graph
display(Image(local_slm_demo.get_graph().draw_mermaid_png()))

#Ask the question
input = [HumanMessage(content=" ماهي الإجراءات المتبعة عند صدور قرار الجمعية العامة غير العادية بتخفيض رأس مال المصدر؟")]


# Run the graph
response = await local_slm_demo.ainvoke({"messages": input})
display(Markdown(f"***Question:*** \n {input[0].content}\n"))
display(Markdown(response['messages'][-1].content))

In [ ]:
# Load packages from the src directory
import sys
from IPython.display import Markdown, display, Image
sys.path.append('../src')

from langchain_core.messages import HumanMessage
from vectrix_graphs import local_slm_demo

#Ask the question
input = [HumanMessage(content="What is the attention mechanism?")]

# Run the graph
response = await local_slm_demo.ainvoke({"messages": input})
display(Markdown(f"***Question:*** \n {input[0].content}\n"))
display(Markdown(response['messages'][-1].content))

In [ ]:
print(response['messages'][-1])

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


human = HumanMessage(content="What is the attention mechanism?")
ai = AIMessage(content="The attention mechanism is a technique used in neural networks to enable the model to focus on relevant parts of the input data during processing.")

messages = ChatPromptTemplate.from_messages([human, ai])


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = messages | llm | StrOutputParser()

chain.invoke({})